In [1]:
%matplotlib inline


In [2]:
import numpy as np
import scipy.stats
from statsmodels.api import GLM, families
from regularized_glm.core import penalized_IRLS

n_samples = 1000
response = scipy.stats.norm(loc=5).rvs(n_samples)
design_matrix = np.ones_like(response)

statsmodels_fit = GLM(response, design_matrix).fit()

coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response)


display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

/Users/edeno/miniconda3/envs/regularized_glm/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


array([4.95360974])

array([4.95360974])

True

array([[0.00103656]])

array([[0.00103656]])

In [3]:
n_samples = 1000
response = np.concatenate(
    (scipy.stats.norm(loc=5).rvs(n_samples // 2),
     scipy.stats.norm(loc=10).rvs(n_samples // 2)))
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

statsmodels_fit = GLM(response, design_matrix).fit()

coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response)


display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

array([5.0162338 , 5.00966818])

array([5.0162338 , 5.00966818])

True

array([[ 0.00209916, -0.00209916],
       [-0.00209916,  0.00419832]])

array([[ 0.00209916, -0.00209916],
       [-0.00209916,  0.00419832]])

In [4]:
n_samples = 1000
response = np.concatenate(
    (scipy.stats.norm(loc=5).rvs(n_samples // 2),
     scipy.stats.norm(loc=10).rvs(n_samples // 2)))
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

statsmodels_fit = GLM(response, design_matrix).fit()

sqrt_penalty_matrix = np.eye(2)
penalty = np.array([0, 1E10])

coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response, penalty=penalty)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

array([5.01499203, 4.99738769])

array([7.51368581e+00, 1.24934689e-07])

True

array([[ 0.00192094, -0.00192094],
       [-0.00192094,  0.00384188]])

array([[7.20922995e-03, 7.20923022e-10],
       [7.20923022e-10, 9.01153789e-17]])

In [5]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones_like(response)

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()
coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response, family=fam)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

array([-4.50986001])

array([-4.50986001])

True

array([[0.04545454]])

array([[0.04545455]])

In [6]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()
coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response, family=fam)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

array([-4.96184513,  0.45198512])

array([-4.96184513,  0.45198512])

True

array([[ 0.14285714, -0.14285714],
       [-0.14285714,  0.23376611]])

array([[ 0.14285714, -0.14285714],
       [-0.14285714,  0.23376623]])

In [7]:
n_samples = 2000
rate = np.ones((n_samples,)) * 12
sampling_frequency = 1000
response = np.random.poisson(rate / sampling_frequency)
design_matrix = np.ones((n_samples, 2))
design_matrix[:n_samples // 2, 1] = 0

fam = families.Poisson()
statsmodels_fit = GLM(response, design_matrix, family=fam).fit()

penalty = np.array([0, 1E3])

coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response, family=fam, penalty=penalty)

display(statsmodels_fit.params)
display(coefficients, is_converged)

display(statsmodels_fit.cov_params())
display(coefficient_covariance)

array([-4.34280592, -0.61903921])

array([-4.60367876e+00, -2.98507464e-03])

True

array([[ 0.07692299, -0.07692299],
       [-0.07692299,  0.21978013]])

array([[4.99809975e-02, 1.00565394e-03],
       [1.00565394e-03, 2.51867306e-05]])

In [9]:
def simulate_time(n_samples, sampling_frequency):
    return np.arange(n_samples) / sampling_frequency


def simulate_linear_distance(time, track_height, running_speed=10):
    half_height = (track_height / 2)
    return (half_height * np.sin(2 * np.pi * time / running_speed - np.pi / 2)
            + half_height)

def simulate_poisson_spikes(rate, sampling_frequency):
    return 1.0 * (np.random.poisson(rate / sampling_frequency) > 0)


def create_place_field(
    place_field_mean, linear_distance, sampling_frequency, is_condition=None,
        place_field_std_deviation=12.5, max_firing_rate=20,
        baseline_firing_rate=0.5):
    if is_condition is None:
        is_condition = np.ones_like(linear_distance, dtype=bool)
    field_firing_rate = scipy.stats.norm(
        place_field_mean, place_field_std_deviation).pdf(linear_distance)
    field_firing_rate /= field_firing_rate.max()
    field_firing_rate[~is_condition] = 0
    return baseline_firing_rate + max_firing_rate * field_firing_rate

In [10]:
from patsy import build_design_matrices, dmatrices, dmatrix

track_height = 170
n_samples = sampling_frequency * 65

time = simulate_time(n_samples, sampling_frequency)
linear_distance = simulate_linear_distance(
    time, track_height)

place_fields = create_place_field(
    100, linear_distance, sampling_frequency)
spikes = simulate_poisson_spikes(place_fields, sampling_frequency).T

knot_spacing = 15

min_distance, max_distance = (linear_distance.min(),
                              linear_distance.max())
n_steps = (max_distance - min_distance) // knot_spacing
position_knots = min_distance + np.arange(1, n_steps) * knot_spacing
formula = ('is_spike ~ 1 + cr(linear_distance, knots=position_knots,'
           ' constraints="center")')

data = {
    'is_spike': spikes,
    'linear_distance': linear_distance
}
response, design_matrix = dmatrices(formula, data)


In [12]:
%%timeit

penalty = np.ones((design_matrix.shape[1],)) * 1E1
penalty[0] = 0.0

coefficients, is_converged, coefficient_covariance, aic, deviance = penalized_IRLS(
    design_matrix, response.squeeze(), family=families.Poisson(), penalty=penalty)

118 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit

model = GLM(response, design_matrix, family=families.Poisson())

penalty = np.ones((design_matrix.shape[1],)) * 1E1
penalty[0] = 0.0
fit = model.fit_regularized(alpha=penalty, L1_wt=0,
                                maxiter=25)

2.2 s ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
